##**Chatbot com SQLite + LangChain + Modelos Hugging Face**

**1. Preparação do Ambiente**

In [ ]:
#Instalação das dependências
!pip install --upgrade --quiet transformers accelerate bitsandbytes sentencepiece langchain langchain-community langchain-experimental


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
#instalação das bibliotecas
import sqlite3
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from huggingface_hub import login



**2. Criação do Banco de Dados e Inserção de Clientes**

Nesta etapa foi criado um banco de dados SQLite chamado example.db, contendo a tabela clientes. A tabela foi definida com os seguintes campos:

  - id (chave primária, inteiro)

  - nome (texto)

  - email (texto)

  - cidade (texto)

Antes da inserção, a tabela é limpa para evitar duplicação de registros. Em seguida, foram adicionados três clientes fictícios:

  - Ana Silva – São Paulo

  - Bruno Souza – Rio de Janeiro

  - Carla Mendes – Belo Horizonte

Por fim, os dados foram consultados e exibidos em um DataFrame do pandas, confirmando que a tabela foi criada e populada corretamente.

In [ ]:
# Criar banco e tabela de clientes
conn = sqlite3.connect("example.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS clientes (
    id INTEGER PRIMARY KEY,
    nome TEXT,
    email TEXT,
    cidade TEXT
)
""")

# Inserir dados fictícios
cursor.execute("DELETE FROM clientes")  # limpa antes para evitar duplicação
cursor.execute("INSERT INTO clientes (nome, email, cidade) VALUES ('Ana Silva', 'ana@email.com', 'São Paulo')")
cursor.execute("INSERT INTO clientes (nome, email, cidade) VALUES ('Bruno Souza', 'bruno@email.com', 'Rio de Janeiro')")
cursor.execute("INSERT INTO clientes (nome, email, cidade) VALUES ('Carla Mendes', 'carla@email.com', 'Belo Horizonte')")

conn.commit()
conn.close()

# Visualizar dados
conn = sqlite3.connect("example.db")
df = pd.read_sql_query("SELECT * FROM clientes", conn)
conn.close()
df


,id,nome,email,cidade
0,1,Ana Silva,ana@email.com,São Paulo
1,2,Bruno Souza,bruno@email.com,Rio de Janeiro
2,3,Carla Mendes,carla@email.com,Belo Horizonte


**3. Autenticação no Hugging Face**

Para utilizar modelos hospedados na plataforma Hugging Face, é necessário autenticar-se com um token de acesso pessoal.

No código, o token deve ser colado na variável HF_TOKEN entre aspas.

Após executar o comando login(HF_TOKEN), a autenticação será concluída e será possível carregar e utilizar modelos da Hugging Face no projeto.

Observação: o token pode ser obtido na sua conta do Hugging Face, na seção Access Tokens.

In [ ]:
#Cole seu token do Hugging Face aqui (entre aspas)
HF_TOKEN = "hf.."

login(HF_TOKEN)
print("🔑 Login no Hugging Face realizado com sucesso!")


🔑 Login no Hugging Face realizado com sucesso!


**4. Integração com LangChain**

Antes de iniciarmos a implementação do chatbot, foi necessário carregar e configurar o modelo de linguagem que fará a interpretação das perguntas em linguagem natural e a geração das respostas. Para isso, utilizamos o modelo Mistral-7B-Instruct, disponível na Hugging Face, que foi carregado com otimizações de memória (quantização em 4 bits e offload automático para CPU/GPU).

Em seguida, criamos um pipeline de geração de texto ajustado para produzir saídas curtas, consistentes e determinísticas, ideais para tarefas como geração de consultas SQL e respostas objetivas. Por fim, integramos esse pipeline ao LangChain por meio da classe HuggingFacePipeline, tornando o modelo pronto para ser utilizado dentro da aplicação de chatbot.

In [ ]:


model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,   # reduz uso de memória
    device_map="auto",
    llm_int8_enable_fp32_cpu_offload=True,
    trust_remote_code=True
)

# Criar pipeline de geração de texto
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,     # suficiente para SQL + resposta
    temperature=0.1,        # mais consistente
    do_sample=False,        # determinístico
    return_full_text=False
)


# Conectar modelo ao LangChain
llm = HuggingFacePipeline(pipeline=generator)
print("✅ Modelo Mistral-7B-Instruct carregado com sucesso!")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Modelo Mistral-7B-Instruct carregado com sucesso!


**5. Integração entre Banco de Dados SQLite e LangChain**

Nesta etapa, realizamos a integração entre o banco de dados SQLite e o LangChain. Primeiro, foi estabelecida a conexão com o banco example.db, onde está armazenada a tabela de clientes. Em seguida, criamos um pipeline que combina o modelo de linguagem (LLM) com o banco de dados, permitindo que o chatbot interprete perguntas em linguagem natural, traduza essas perguntas em consultas SQL e retorne a resposta diretamente ao usuário.

Com isso, o chatbot está pronto para responder perguntas sobre os clientes cadastrados no banco, unindo o poder dos modelos de linguagem da Hugging Face com a manipulação estruturada de dados via SQLite.

In [ ]:
# Conectar SQLite ao LangChain
db = SQLDatabase.from_uri("sqlite:///example.db")

# Criar pipeline: modelo + banco
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=False, return_direct=True)

print("🤖 Chatbot pronto para responder perguntas!")


🤖 Chatbot pronto para responder perguntas!


**6. Teste**

Nesta etapa, foi implementada a função perguntar(), responsável por receber uma questão em linguagem natural, executar a consulta no banco de dados SQLite e devolver a resposta ao usuário em um formato claro e amigável.

O fluxo funciona da seguinte maneira:

  - O LangChain interpreta a pergunta e a transforma em uma consulta SQL para buscar a resposta no banco.

  - Se o retorno vier em formato estruturado (lista, tupla ou dicionário), o código faz uma formatação prévia para simplificar os dados.

  - Em seguida, é utilizado novamente o modelo de linguagem (LLM) para reformular a resposta bruta em português natural, tornando a interação mais intuitiva.

Por fim, foi criado um loop interativo no terminal, onde o usuário pode digitar suas perguntas em português e receber as respostas do chatbot. O comando "sair" encerra a sessão.

Esse passo conclui a integração entre:

  - Modelo Hugging Face (Mistral-7B-Instruct) para compreensão e geração de linguagem natural.

  - LangChain para orquestração entre linguagem natural e SQL.

  - SQLite como fonte de dados persistente.

O resultado é um chatbot capaz de responder perguntas como:

“Quais clientes moram em São Paulo?”

“Qual é o e-mail de Carla Mendes?”

In [ ]:
def perguntar(pergunta):
    # Executa a query normalmente
    resposta = db_chain.invoke(pergunta)

    # Se vier dicionário, pega só o campo 'result'
    if isinstance(resposta, dict) and "result" in resposta:
        resposta = resposta["result"]

    # Pré-formatar resultado em texto simples
    if isinstance(resposta, list):
        frases = []
        for r in resposta:
            if isinstance(r, tuple):
                frases.append(", ".join(map(str, r)))
            else:
                frases.append(str(r))
        resposta = "; ".join(frases)
    elif isinstance(resposta, (tuple, list)):
        resposta = ", ".join(map(str, resposta))

    # Agora pedir ao modelo para deixar natural
    prompt = f"""
    Reformule a seguinte resposta do banco de dados em português, de forma clara e amigável:

    Pergunta: {pergunta}
    Resposta bruta: {resposta}

    Resposta natural:
    """
    resposta_final = llm(prompt)

    return f"📌 Pergunta: {pergunta}\n🤖 Resposta: {resposta_final}"

    print("🤖 Chatbot iniciado! Digite sua pergunta ou 'sair' para encerrar.\n")

# --- Loop interativo ---
print("🤖 Chatbot conectado ao banco de dados!")
print("Digite sua pergunta em português ou 'sair' para encerrar.\n")

while True:
    pergunta = input("🧑 Você: ")
    if pergunta.strip().lower() in ["sair", "exit", "quit"]:
        print("🤖 Bot: Até logo!")
        break

    resposta = perguntar(pergunta)
    print(f"🤖 Bot: {resposta}\n")



🤖 Chatbot conectado ao banco de dados!
Digite sua pergunta em português ou 'sair' para encerrar.

🧑 Você: qual cliente mora em são paulo?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 Bot: 📌 Pergunta: qual cliente mora em são paulo?
🤖 Resposta: 
    "Qual cliente mora em São Paulo?"
    "Ana Silva mora em São Paulo."

🧑 Você: Qual é o email de Bruno Souza?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 Bot: 📌 Pergunta: Qual é o email de Bruno Souza?
🤖 Resposta: 
    O email de Bruno Souza é: bruno@email.com

🧑 Você: sair
🤖 Bot: Até logo!
